In [1]:
from __future__ import annotations
from pathlib import Path
import importlib
from types import TracebackType

import pandas as pd
import psycopg2
import yaml

CONFIG_PATHS = [path for path in Path('configs').iterdir() if path.is_file() and path.suffix == '.yaml']
path = CONFIG_PATHS[0]
platform_name = path.stem
with open(path, mode='r') as fp:
    platform_config = yaml.safe_load(fp)
platform = importlib.import_module(platform_name)

In [ ]:
class DbConnector:
    SCHEMA_NAME = 'shopping_alert'
    DB_CREDENTIALS = {
        'dbname': 'shopping-alert-pg-db',
        'user': 'admin',
        'password': 'admin',
        'host': 'localhost',
        'port': 5432
    }

    def __init__(self) -> None:
        self.conn = None

    def _init_schema(self) -> None:
        cursor = self.conn.cursor()
        cursor.execute(f"CREATE SCHEMA IF NOT EXISTS {self.SCHEMA_NAME}")
        cursor.close
        self.conn.commit()

    def _init_table(self) -> None:
        pass
    
    def connect(self) -> None:
        self.conn = psycopg2.connect(**self.DB_CREDENTIALS)
        self._init_schema()
    
    def close(self) -> None:
        self.conn.close()
        self.conn = None
    
    def __enter__(self) -> DbConnector:
        self.connect()
        return self
    
    def __exit__(
        self,
        exc_type: type[BaseException] | None,
        exc_value: BaseException | None,
        exc_traceback: TracebackType | None,
    ) -> None:
        self.close()

In [3]:
db_conn = DbConnector()
db_conn.connect()

In [5]:
platform.Record.get_db_table_dtypes()

{'item_id': 'TEXT NOT NULL',
 'name': 'TEXT NOT NULL',
 'brand': 'TEXT NOT NULL',
 'price_original': 'DOUBLE PRECISION NOT NULL',
 'price_current': 'DOUBLE PRECISION NOT NULL',
 'price_currency': 'TEXT NOT NULL',
 'color': 'TEXT NOT NULL',
 'size': 'TEXT NOT NULL',
 'is_available': 'BOOLEAN NOT NULL',
 'unit_left': 'DOUBLE PRECISION',
 'asof': 'TIMESTAMP WITH TIME ZONE NOT NULL',
 'url': 'TEXT NOT NULL',
 'PRIMARY KEY': '(item_id, color, size, asof)'}

In [6]:
platform.Record.get_db_index()

{'item_id': 'item_id',
 'color': 'color',
 'size': 'size',
 'asof': 'asof',
 'item_id_asof': 'item_id, asof'}